In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv("./fer2013.csv")

In [ ]:
data

In [ ]:
width,height=48,48
datapoints=data['pixels'].tolist()

In [ ]:
x=[]
for i in datapoints:
    xx=[int(xp) for xp in i.split(' ')]
    xx=np.asarray(xx).reshape(width,height)
    x.append(xx.astype('float32'))


In [ ]:
x=np.asarray(x)
x

In [ ]:
x.shape

In [ ]:
x=np.expand_dims(x,-1)
x.shape

In [ ]:
y=pd.get_dummies(data['emotion']).as_matrix()

In [ ]:
np.save('fdatax',x)
np.save('flabels',y)

In [ ]:
import sys, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2

In [ ]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 25
width, height = 48, 48

In [ ]:
x = np.load('./fdataX.npy')
y = np.load('./flabels.npy')

x -= np.mean(x, axis=0)
x /= np.std(x, axis=0)

In [ ]:
import matplotlib.pyplot as plt
for xx in range(10):
    plt.figure(xx)
    plt.imshow(x[xx].reshape((48, 48)), interpolation='none', cmap='gray')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41)
np.save('modXtest', X_test)
np.save('modytest', y_test)

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(2*num_features, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_features, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

In [ ]:
model.compile(loss=categorical_crossentropy,optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),metrics=['accuracy'])

In [ ]:
model.fit(np.array(X_train), np.array(y_train),batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(np.array(X_valid), np.array(y_valid)),shuffle=True)

In [ ]:
fer1_json = model.to_json()
with open("fer1.json", "w") as json_file:
    json_file.write(fer1_json)
model.save_weights("fer1.h5")

In [ ]:
from keras.models import model_from_json
json_file = open('fer1.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#load weights into new model
loaded_model.load_weights("fer1.h5")
print("Loaded Model from disk")

#compile and evaluate loaded model
loaded_model.compile(loss=categorical_crossentropy,optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),metrics=['accuracy'])

In [ ]:
y_predict=loaded_model.predict(X_test)

In [ ]:
from sklearn import metrics

In [ ]:
y1=[]
yy1=[]
for i in range(0,3589):
    z=0
    zz=0
    z1=-1
    zz1=-1
    for j in range(0,7):
        if z<y_predict[i][j]:
            z=y_predict[i][j]
            z1=j+1
        if zz<y_test[i][j]:
            zz=y_test[i][j]
            zz1=j+1
    y1.append(z1)
    yy1.append(zz1)

In [ ]:
print("Accuracy:",metrics.accuracy_score(yy1, y1)*100.0)